# Differential Evolution
## Implementation of the DE optimization algorithm

In [1]:
from functools import *
import numpy as np
import random
from tqdm import tqdm

In [4]:
class DE:
    """Differential Evolution optimizer"""
    
    def __init__(self, eval_func,
                 dim=2, 
                 inf=0, 
                 sup=100, 
                 n_agents=30, 
                 n_cycles=500, 
                 CR=0.9, 
                 F=0.8,
                 N_NEIGHBORS=3,
                 isbetter_func=lambda f1, f2: f1 < f2,
                 bounding_func=None,
                 init_func=None,
                 log_func=None):
        self.dim = dim
        self.inf = inf
        self.sup = sup
        self.n_agents = n_agents
        self.n_cycles = n_cycles
        self.CR = CR
        self.F = F
        self.N_NEIGHBORS = N_NEIGHBORS
        
        self.eval_func = eval_func
        self.isbetter_func = isbetter_func
        self.bounding_func = bounding_func if bounding_func is not None else self.bounding
        self.init_func = init_func if init_func is not None else self.plain_init
        self.log_func = log_func if log_func is not None else self.display_result
        
        self.population = []
        self.best_agent = None

    def init_population(self):
        positions = self.init_func()
        fits = [self.eval_func(pos) for pos in positions]
        self.population = [{'pos': positions[i], 'fit': fits[i]} for i in range(self.n_agents)]
        return self.population

    def get_best_agent(self):
        best_agent = self.population[0]
        for agent in self.population[1:]:
            if self.isbetter_func(agent['fit'],best_agent['fit']):
                best_agent = agent
        self.best_agent = best_agent
        return self.best_agent

    def update_agent(self, agent):
        new_agent = dict(agent)
        temp_population = list(filter(lambda a: a is not agent, self.population))
        # random selection of 3 individuals, distincts of themselves and of the currently concerned agent
        a,b,c = random.choices(temp_population, k=self.N_NEIGHBORS)
        # ensured component heritance (index)
        index = random.choice(list(range(self.dim))) 

        new_pos = [0] * self.dim
        for i in range(self.dim):
            ri = random.uniform(0,1)
            if ri < self.CR or i == index: # second condition to ensure that at least one component of the vector is herited
                new_pos[i] = a['pos'][i] + self.F * (b['pos'][i] - c['pos'][i])
            else:
                new_pos[i] = new_agent['pos'][i]
        new_eval = self.eval_func(new_pos)
        if self.isbetter_func(new_eval, new_agent['fit']):
            try:
                new_pos = self.bounding_func(new_pos)
            except ValueError:
                new_pos = new_agent['pos']
                new_eval = new_agent['fit']
            new_agent['pos'] = new_pos[:]
            new_agent['fit'] = new_eval

        return new_agent

    def bounding(self, position):
        return [min(self.sup, max(self.inf, dim_val)) for dim_val in position]

    def plain_init(self):
        return [[random.uniform(self.inf, self.sup) for i in range(self.dim)] for a in range(self.n_agents)]
    
    def fit(self):
        self.init_population()
        self.log_func(map(lambda s: s["pos"], self.population), colors=["green", "purple"], legend="DE - Initialization", padding=1)
        best_global = self.get_best_agent()
        best_local = best_global

        for i in tqdm(range(self.n_cycles)):
            self.population = [self.update_agent(agent) for agent in self.population]
            self.log_func(map(lambda s: s["pos"], self.population), colors=["green", "purple"], legend="DE - Swarm", it=i+1)
            best_local = self.get_best_agent()
            if self.isbetter_func(best_local["fit"], best_global["fit"]):
                best_global = best_local
                
            # historization of data
            self.log_func(best_global['pos'], legend="DE - Best", text=f"{int(best_global['fit'])}", colors=["green", "blue"], it=i+1, fitness=best_global['fit'])
        
        self.best_agent = best_global
        self.log_func(best_global['pos'], legend="DE - Best final", text=f"{int(best_global['fit'])}", colors=["green", "blue"], it=self.n_cycles, fitness=best_global['fit'])
        
        
    def display_result(self, sols, colors=None, legend="", text="", it=0, fitness=None, **kwargs):
        if legend.split(" - ")[-1].startswith("Best") and it % (self.n_cycles / 5) == 0:
            print("Itération", it)
            print(f"{legend} : {text}")
            print(sols)
            if fitness is not None:
                print("fitness =", fitness)
            print("--------------------\n")

In [5]:
if __name__ == "__main__":
    
    def sphere(x):
        return reduce(lambda acc,e: acc + e * e, x, 0)
    
    optimizer = DE(eval_func=sphere, dim=5, inf=-50, sup=50)
    optimizer.fit()

100%|██████████| 500/500 [00:00<00:00, 3216.66it/s]

Itération 100
DE - Best : 0
[0.005094282525460542, -0.065425743226922, -0.02410552642485006, -0.008874561688646573, -0.017965885598548934]
fitness = 0.005289086885969229
--------------------

Itération 200
DE - Best : 0
[-8.694738663084422e-06, 1.019405664321466e-05, 4.5695414970078836e-05, -2.444380493452864e-05, -4.324733773248578e-05]
fitness = 4.735420041177084e-09
--------------------

Itération 300
DE - Best : 0
[5.0219466019443394e-08, 7.9329679238741e-09, -4.810016699460227e-09, -3.121870552590459e-08, 5.7967895573486935e-08]
fitness = 6.9429474999401635e-15
--------------------

Itération 400
DE - Best : 0
[-6.977449374728035e-11, -4.676772561853354e-11, 5.459329993456002e-11, 6.419126243927829e-12, -3.747341921589059e-11]
fitness = 1.1481590864429896e-20
--------------------

Itération 500
DE - Best : 0
[-3.848017885289395e-15, 7.334034027000428e-14, -1.0293591519588257e-13, 1.3564294421814697e-14, 4.5244851403810434e-14]
fitness = 1.8220502051494006e-26
--------------------
